This notebook is for Figure 6

In [1]:

#%matplotlib inline
import pickle
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
import tensorflow as tf
import sys, os
from fnmatch import fnmatch
from keras.models import Sequential 
from keras.layers import Activation, Dense, Flatten,Conv1D, MaxPooling1D,Dropout,GlobalAveragePooling1D

from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import InputLayer
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import concatenate

In [2]:
# to get all .pkl files in a base directory. It will go to every sub directory find the .pkl file

signals = [] # it will store all EDA signals of all subjects from "chest" device
root = "Dataset/WESAD/untitled folder/"
pattern = "*.pkl"
signal_name='EDA'

file_paths = glob.glob(root+pattern)
subs=[]
for path in  file_paths:
    print(path)
    with open(path, 'rb') as file:
        data = pickle.load(file, encoding='latin1')
    eda_signal= data['signal']['chest'][signal_name][:,0]
    label=data['label']
    numpy_data1=np.array([eda_signal,label])
    numpy_data1=numpy_data1.T
    df = pd.DataFrame(data=numpy_data1, columns=[signal_name,"Label"]) 
    df= df[df["Label"]==1].reset_index(drop=True)
    signals.append(df[signal_name].values)
            

In [4]:
import random
def delete_random_elems(input_list, n):
    to_delete = set(random.sample(range(len(input_list)), n))
    return [x for i,x in enumerate(input_list) if not i in to_delete]

In [5]:
signal_same_length = []

for signal in signals:
    if len(signal)==798000:
        signal_same_length.append(signal)
    else:
        signal_same_length.append(np.array(delete_random_elems(signal, len(signal)-798000)))

In [ ]:
subjects=[x.split('/')[-1].split('.')[0] for x in file_paths]

In [6]:
def evaluate_model_pretrain(subject,trainX,trainy,testX,testy,question,model,signal_name,data_point,step):
    trainX,trainy,testX,testy=np.array(trainX),np.array(trainy),np.array(testX),np.array(testy)
    
    verbose, epochs, batch_size = 0, 100, 2000
    
    model.compile(loss="mean_squared_error", optimizer='adam', metrics=['mean_squared_error'])

    history=model.fit(np.array(trainX), np.array(trainy), validation_split=0.1,epochs=60, batch_size=1000, verbose=0)
    _,accuracy = model.evaluate(np.array(testX), np.array(testy),batch_size=1000,verbose=0)
    accuracy = math.sqrt(abs(accuracy))

    df=pd.DataFrame({'Subject':[subject],
                     'Data point':[data_point],
                     'Step':[step],
                     'Type':["Self-Supervised Pretraining"],
                     'Question': [question],
                     'SignAl':[signal],
                    'RMSE':[accuracy]})
    print(accuracy)
    return df
    
    
    
    

In [3]:
def evaluate_model_supervised(trainX,trainy,testX,testy,subject,signal,question,data_point,step):
    trainX,trainy,testX,testy=np.array(trainX),np.array(trainy),np.array(testX),np.array(testy)
    
    verbose, epochs, batch_size = 1, 100, 1000
    model = tf.keras.models.Sequential()
    model.add(Conv1D(40, 4, activation='leaky_relu',padding="same", input_shape=(trainX.shape[1],1)))

    model.add(Conv1D(30, 2,padding="same", activation='leaky_relu'))

    model.add(Conv1D(18, 4,padding="same", activation='leaky_relu'))

    model.add(Conv1D(30, 2,padding="same", activation='leaky_relu'))

    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(70, activation='leaky_relu'))
    model.add(Dense(30, activation='leaky_relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss="mean_squared_error",optimizer='adam', metrics=['mean_squared_error'])

    

    
    history=model.fit(np.array(trainX), np.array(trainy), validation_split=0.1,epochs=60, batch_size=2000, verbose=0)
    _,accuracy = model.evaluate(np.array(testX), np.array(testy),batch_size=1000,verbose=0)
    accuracy = math.sqrt(abs(accuracy))

    
    df=pd.DataFrame({'Subject':[subject],
                     'Data point':[data_point],
                     'Step':[step],
                     'Type':["Supervised"],
                     'Question': [question],
                     'SignAl':[signal],
                    'RMSE':[accuracy]})
    print(accuracy)
    return df
    
    
    
    

In [8]:
def pretrain_model_selection(sub):
    res=pd.read_csv("Models/PretrainModelsWithForecasting/"+"EDA/"+sub+"/30Seconds_"+str(sub)+"_mae.csv")
    res.sort_values(by=['MAE'],ascending=True, inplace=True)
    model_no= res['Model No'].iloc[0]

    import keras
    model=keras.models.load_model("Models/PretrainModelsWithForecasting/"+"EDA"+"/"+"S"+str(int(sub[1:]))+"/30Seconds_"+str(model_no)+"_model.h5")
    inp = keras.layers.Input(shape=(798000,1), name='image_input')
    for i in range(0,6):
        model.layers[i].trainable = False



    model.trainable = False
    names=[]
    for layer in model.layers:
        names.append(layer.name)
    getLayer=names[5]
    last_layer = model.get_layer(getLayer).output
    out = Dense(40, activation='leaky_relu')(last_layer)
    out = Dense(30, activation='leaky_relu')(out)
    out = Dense(10, activation='leaky_relu')(out)
    out = Dense(1, activation='linear')(out)
    new_model=Model(model.input, out)
    return new_model




In [9]:
def build_data_with_forecasting(signal,window,output_length, overlap):
    X=[]
    y=[]
    start=0
    while (True):
        x1=signal[start:start+window]
        output=signal[start+window:start+window+output_length] # 10 points
        if (len(output)!=output_length):
            break
        X.append(x1)
        y.append(output)
        start=start+overlap
    return X

In [ ]:
window=7000
output_length = 40
overlap = 100

data_points=[10] 
signal="EDA"

# signal_same_length=signal_same_length[7:]
# subjects = ['S6']
subject_no=1
for subject in range (0,15) :
    


#     subject_no=1
    for question in range(0,6):
        evaluation_list_pretrain=[]
        evaluation_list_supervised=[]
        y_new=[]
        y=[]
        with open('Dataset/QuestLabels/'+subjects[subject]+'.pickle', 'rb') as handle:
            b = pickle.load(handle)
            y.append(b['values']['Base']['stai'][question])
        y=pd.DataFrame(y)
        y[0]= y[0].map({1:0.25,2:0.5,3:0.75,4:1})
        y = list(y[0].values) 
        for x in range(0,7910): # copy same output value as target to all  signals for. here total generated overlap signal 7910
            y_new.append(y)
#         for i in range(0,len(subjects)):
        print("Subject: ",subject)
        print("Question: ",question+1)
        X_new= build_data_with_forecasting(signal_same_length[subject_no-1],window,output_length, overlap)
        
        X=X_new.copy()
        y=y_new.copy()
        testX=X[7000:]
        testy=y[7000:]
        for data_point in data_points:
            for step in [1,2,3,4,5]:
                print("Data Point: ",data_point,end=" ")
                print("Step: ",step)
                randomlist = random.sample(range(0, 7000), data_point) # Here 910 data points is for testing. thats why 7910-910=7000
                trainX= [X[i] for i in randomlist] 
                trainy=[y[i] for i in randomlist]
                new_model = pretrain_model_selection(subjects[subject])
                eval_model_pretrain = evaluate_model_pretrain(subjects[subject],trainX,trainy,testX,testy,question+1,new_model,signal,data_point,step)
                eval_model_supervised = evaluate_model_supervised(trainX,trainy,testX,testy,subjects[subject],signal,question+1,data_point,step)

                evaluation_list_pretrain.append(eval_model_pretrain)
                evaluation_list_supervised.append(eval_model_supervised)
        df_pretrain=pd.concat(evaluation_list_pretrain)
        df_supervised=pd.concat(evaluation_list_supervised)
        df_pretrain.to_csv("Models/ClassificationUsingBoostrap/EDA/"+subjects[subject]+"/Pretrain_Question_"+str(question)+".csv",index=False)
        df_supervised.to_csv("Models/ClassificationUsingBoostrap/EDA/"+subjects[subject]+"/Supervised_Question_"+str(question)+".csv",index=False)
        
    subject_no=subject_no+1
        
    